In [1]:
import pandas as pd
import unicodedata
import re
from difflib import get_close_matches

In [2]:
stats_2024 = pd.read_csv('../general/stats/2024_batting_stats_edited.csv')
stats_2025 = pd.read_csv('../general/stats/2025_batting_stats_edited.csv')
def_rank_2024 = pd.read_csv('../general/stats/2024/def_rank_era_pitchers_2024.csv')
def_rank_2025=pd.read_csv('../scrapers/def_scraper/batter_runs_def_rank.csv')
batter_team_2024 = pd.read_csv('../general/stats/2024/batters_team_2024.csv')
batter_team_2025 = pd.read_csv('../general/stats/batter_team_2025.csv')

In [3]:
stats_2024['games_played'] = 1
stats_2025['games_played'] = 1

In [4]:
stats_2025

,Unnamed: 0,away_hitters,away_AB,away_R,away_H,away_RBI,away_HR,away_BB,away_K,away_AVG,...,home_K,home_AVG,home_OBP,home_SLG,away_player,away_team,home_player,home_team,gameID,games_played
0,0,F. LindorSS,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.293,...,0.0,0.257,0.357,0.544,Francisco Lindor,NYM,Ben Rice,NYY,1,1
1,1,J. SotoRF,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.255,...,1.0,0.409,0.495,0.766,Juan Soto,NYM,Aaron Judge,NYY,1,1
2,2,P. Alonso1B,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.305,...,1.0,0.238,0.304,0.397,Pete Alonso,NYM,Cody Bellinger,NYY,1,1
3,3,B. NimmoLF,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.230,...,1.0,0.345,0.393,0.500,Brandon Nimmo,NYM,Paul Goldschmidt,NYY,1,1
4,4,M. VientosDH,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.243,...,0.0,0.240,0.336,0.426,Mark Vientos,NYM,Jasson Dominguez,NYY,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,7608,L. Urias2B,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.259,...,0.0,0.216,0.263,0.443,Luis Urias,ATH,Ryan O'Hearn,BAL,630,1
7609,7609,N. Kurtz1B,5.0,2.0,2.0,1.0,1.0,0.0,1.0,0.281,...,1.0,0.172,0.303,0.287,Nick Kurtz,ATH,Michael Conforto,LAD,630,1
7610,7610,team,43.0,11.0,18.0,11.0,5.0,1.0,11.0,NaN,...,0.0,0.250,0.274,0.333,NaN,ATH,Miguel Rojas,LAD,630,1
7611,7611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,0.219,0.219,0.281,NaN,ATH,Chris Taylor,LAD,630,1


In [5]:
away_teams_2024=stats_2024[['away_player','away_R','home_team','games_played','away_AB','away_BB','away_K']]
away_teams_2025=stats_2025[['away_player','away_R','home_team','games_played','away_AB','away_BB','away_K']]

In [6]:
home_teams_2024=stats_2024[['home_player','home_R','away_team','games_played','home_AB','home_BB','home_K']]
home_teams_2025=stats_2025[['home_player','home_R','away_team','games_played','home_AB','home_BB','home_K']]

In [7]:
away_teams_2024.dropna()
away_teams_2025.dropna()
home_teams_2024.dropna()
home_teams_2025.dropna()

,home_player,home_R,away_team,games_played,home_AB,home_BB,home_K
0,Ben Rice,0.0,NYM,1,2.0,0.0,0.0
1,Aaron Judge,0.0,NYM,1,2.0,0.0,1.0
2,Cody Bellinger,0.0,NYM,1,2.0,0.0,1.0
3,Paul Goldschmidt,0.0,NYM,1,2.0,0.0,1.0
4,Jasson Dominguez,0.0,NYM,1,2.0,0.0,0.0
...,...,...,...,...,...,...,...
7607,Andy Pages,0.0,ATH,1,4.0,0.0,1.0
7608,Ryan O'Hearn,0.0,ATH,1,4.0,0.0,0.0
7609,Michael Conforto,0.0,ATH,1,4.0,0.0,1.0
7610,Miguel Rojas,0.0,ATH,1,4.0,0.0,0.0


In [8]:
# Convert both columns to numeric, coercing errors to NaN
away_teams_2024[['away_AB','away_BB']] = away_teams_2024[['away_AB','away_BB']].apply(
    pd.to_numeric, errors='coerce'
)
# Convert both columns to numeric, coercing errors to NaN
away_teams_2025[['away_AB','away_BB']] = away_teams_2025[['away_AB','away_BB']].apply(
    pd.to_numeric, errors='coerce'
)
# Convert both columns to numeric, coercing errors to NaN
home_teams_2024[['home_AB','home_BB']] = home_teams_2024[['home_AB','home_BB']].apply(
    pd.to_numeric, errors='coerce'
)
# Convert both columns to numeric, coercing errors to NaN
home_teams_2025[['home_AB','home_BB']] = home_teams_2025[['home_AB','home_BB']].apply(
    pd.to_numeric, errors='coerce'
)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_67960\605779913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_teams_2024[['away_AB','away_BB']] = away_teams_2024[['away_AB','away_BB']].apply(
C:\Users\Joshua\AppData\Local\Temp\ipykernel_67960\605779913.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_teams_2025[['away_AB','away_BB']] = away_teams_2025[['away_AB','away_BB']].apply(
C:\Users\Joshua\AppData\Local\Temp\ipykernel_67960\605779913.py:10: SettingWithCopyWarning: 
A value is trying t

In [9]:
# require at least 2 plate appearances (AB + BB >= 2)
away_teams_2024 = away_teams_2024[
    (away_teams_2024['away_AB'] + away_teams_2024['away_BB']) >= 2
].reset_index(drop=True)
away_teams_2025 = away_teams_2025[
    (away_teams_2025['away_AB'] + away_teams_2025['away_BB']) >= 2
].reset_index(drop=True)

In [10]:
# Version 1: require at least 3 plate appearances (AB + BB >= 2)
home_teams_2024 = home_teams_2024[
    (home_teams_2024['home_AB'] + home_teams_2024['home_BB']) >= 2
].reset_index(drop=True)
home_teams_2025 = home_teams_2025[
    (home_teams_2025['home_AB'] + home_teams_2025['home_BB']) >= 2
].reset_index(drop=True)


In [11]:
away_teams_2024=stats_2024[['away_player','away_R','home_team','games_played']]
away_teams_2025=stats_2025[['away_player','away_R','home_team','games_played']]

In [12]:
home_teams_2024=stats_2024[['home_player','home_R','away_team','games_played']]
home_teams_2025=stats_2025[['home_player','home_R','away_team','games_played']]

In [13]:
#dropping first half of the 2024 season
# stats_2024 = stats_2024.iloc[len(stats_2024)//2:]

In [14]:
team_mapping = {
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'Chicago Cubs': 'CHC',
    'Colorado Rockies': 'COL',
    'Washington Nationals': 'WSH',
    'Toronto Blue Jays': 'TOR',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'New York Yankees': 'NYY',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'Kansas City Royals': 'KCR',
    'Arizona Diamondbacks': 'ARI',
    'Boston Red Sox': 'BOS',
    'Detroit Tigers': 'DET',
    'Cleveland Guardians': 'CLE',
    'Tampa Bay Rays': 'TBR',
    'San Francisco Giants': 'SFG',
    'Los Angeles Dodgers': 'LAD',
    'Seattle Mariners': 'SEA',
    'Cincinnati Reds': 'CIN',
    'Texas Rangers': 'TEX',
    'Chicago White Sox': 'CHW',
    'Los Angeles Angels': 'LAA',
    'Miami Marlins': 'MIA',
    'New York Mets': 'NYM',
    'Athletics': 'ATH',
    'St. Louis Cardinals': 'STL',
    'San Diego Padres': 'SDP',
    'Houston Astros': 'HOU'
}

def_rank_2024['team'] = def_rank_2024['team'].map(team_mapping)
def_rank_2025['Team'] = def_rank_2025['Team'].map(team_mapping)

In [15]:
#merge df_rank_2024 into merged_df_2 to bring in the def_rank 
merged_df_away_2024 = away_teams_2024.merge(
    def_rank_2024,
    how='left',
    left_on='home_team',
    right_on='team'
)
merged_df_away_2024 = merged_df_away_2024.drop(columns=['team'])

In [16]:
merged_df_away_2024=merged_df_away_2024[['away_player','away_R','home_team','Rank','games_played']]

In [17]:
#merge df_rank_2024 into merged_df_2 to bring in the def_rank 
merged_df_home_2024 = home_teams_2024.merge(
    def_rank_2024,
    how='left',
    left_on='away_team',
    right_on='team'
)
merged_df_home_2024 = merged_df_home_2024.drop(columns=['team'])

In [18]:
merged_df_home_2024=merged_df_home_2024[['home_player','home_R','away_team','Rank','games_played']]

In [19]:
#merge df_rank_2025 into merged_df_2 to bring in the def_rank 
merged_df_away_2025 = away_teams_2025.merge(
    def_rank_2025,
    how='left',
    left_on='home_team',
    right_on='Team'
)
merged_df_away_2025 = merged_df_away_2025.drop(columns=['Team'])

In [20]:
merged_df_away_2025=merged_df_away_2025[['away_player','away_R','home_team','Rank','games_played']]

In [21]:
#merge df_rank_2024 into merged_df_2 to bring in the def_rank 
merged_df_home_2025 = home_teams_2025.merge(
    def_rank_2025,
    how='left',
    left_on='away_team',
    right_on='Team'
)
merged_df_home_2025 = merged_df_home_2025.drop(columns=['Team'])

In [22]:
merged_df_home_2025=merged_df_home_2025[['home_player','home_R','away_team','Rank','games_played']]

In [23]:
merged_df_home_2024 = merged_df_home_2024.dropna(subset=['home_player'])
merged_df_away_2024 = merged_df_away_2024.dropna(subset=['away_player'])
merged_df_home_2025 = merged_df_home_2025.dropna(subset=['home_player'])
merged_df_away_2025 = merged_df_away_2025.dropna(subset=['away_player'])


In [24]:
merged_df_home_2025

,home_player,home_R,away_team,Rank,games_played
0,Ben Rice,0.0,NYM,1.0,1
1,Aaron Judge,0.0,NYM,1.0,1
2,Cody Bellinger,0.0,NYM,1.0,1
3,Paul Goldschmidt,0.0,NYM,1.0,1
4,Jasson Dominguez,0.0,NYM,1.0,1
...,...,...,...,...,...
7607,Andy Pages,0.0,ATH,26.0,1
7608,Ryan O'Hearn,0.0,ATH,26.0,1
7609,Michael Conforto,0.0,ATH,26.0,1
7610,Miguel Rojas,0.0,ATH,26.0,1


In [25]:
#renaming the columns of the away players 
merged_df_away_2024 = merged_df_away_2024.rename(columns={
    'away_player': 'player',
    'home_team': 'team',
    'away_R': 'runs',
    'Rank': 'def_rank'
})

In [26]:
#renaming the columns of the away players 
merged_df_home_2024 = merged_df_home_2024.rename(columns={
    'home_player': 'player',
    'away_team': 'team',
    'home_R': 'runs',
    'Rank': 'def_rank'
})

In [27]:
#renaming the columns of the away players 
merged_df_away_2025 = merged_df_away_2025.rename(columns={
    'away_player': 'player',
    'home_team': 'team',
    'away_R': 'runs',
    'Rank': 'def_rank'
})

In [28]:
#renaming the columns of the away players 
merged_df_home_2025 = merged_df_home_2025.rename(columns={
    'home_player': 'player',
    'away_team': 'team',
    'home_R': 'runs',
    'Rank': 'def_rank'
})

In [29]:
# Convert 'hits' to numeric (errors='coerce' turns bad values into NaN)
merged_df_away_2024['runs'] = pd.to_numeric(merged_df_away_2024['runs'], errors='coerce')
merged_df_home_2024['runs'] = pd.to_numeric(merged_df_home_2024['runs'], errors='coerce')
merged_df_away_2025['runs'] = pd.to_numeric(merged_df_away_2025['runs'], errors='coerce')
merged_df_home_2025['runs'] = pd.to_numeric(merged_df_home_2025['runs'], errors='coerce')

In [30]:
#putting 1s in to indicate if they got 1 or 2 hits based off the column
merged_df_away_2024['1_runs_count'] = (merged_df_away_2024['runs'] > 0).astype(int)
merged_df_away_2024['2_runs_count'] = (merged_df_away_2024['runs'] > 1.5).astype(int)
merged_df_away_2024['3_runs_count'] = (merged_df_away_2024['runs'] > 2.5).astype(int)
merged_df_away_2024['4_runs_count'] = (merged_df_away_2024['runs'] > 3.5).astype(int)

merged_df_home_2024['1_runs_count'] = (merged_df_home_2024['runs'] > 0).astype(int)
merged_df_home_2024['2_runs_count'] = (merged_df_home_2024['runs'] > 1.5).astype(int)
merged_df_home_2024['3_runs_count'] = (merged_df_home_2024['runs'] > 2.5).astype(int)
merged_df_home_2024['4_runs_count'] = (merged_df_home_2024['runs'] > 3.5).astype(int)


merged_df_away_2025['1_runs_count'] = (merged_df_away_2025['runs'] > 0).astype(int)
merged_df_away_2025['2_runs_count'] = (merged_df_away_2025['runs'] > 1.5).astype(int)
merged_df_away_2025['3_runs_count'] = (merged_df_away_2025['runs'] > 2.5).astype(int)
merged_df_away_2025['4_runs_count'] = (merged_df_away_2025['runs'] > 3.5).astype(int)


merged_df_home_2025['1_runs_count'] = (merged_df_home_2025['runs'] > 0).astype(int)
merged_df_home_2025['2_runs_count'] = (merged_df_home_2025['runs'] > 1.5).astype(int)
merged_df_home_2025['3_runs_count'] = (merged_df_home_2025['runs'] > 2.5).astype(int)
merged_df_home_2025['4_runs_count'] = (merged_df_home_2025['runs'] > 3.5).astype(int)

In [31]:
#merging one row from df and then the next row from df2 and back and fourth
# Make sure both DataFrames have the same columns

merged_df_away_2024 = merged_df_away_2024.reset_index(drop=True)
merged_df_home_2024 = merged_df_home_2024.reset_index(drop=True)
merged_df_away_2025 = merged_df_away_2025.reset_index(drop=True)
merged_df_home_2025 = merged_df_home_2025.reset_index(drop=True)


# Stack them row by row
interleaved_2024 = pd.concat([merged_df_away_2024, merged_df_home_2024]).sort_index(kind='merge').reset_index(drop=True)
interleaved_2025 = pd.concat([merged_df_away_2025, merged_df_home_2025]).sort_index(kind='merge').reset_index(drop=True)

In [32]:
#unders are only against top 15 teams and overs are only against bot 15 teams, so we split then to get the overs and unders df
unders_2024 = interleaved_2024[interleaved_2024['def_rank'] < 16].reset_index(drop=True)
overs_2024 = interleaved_2024[interleaved_2024['def_rank'] > 15].reset_index(drop=True)
unders_2025 = interleaved_2025[interleaved_2025['def_rank'] < 16].reset_index(drop=True)
overs_2025 = interleaved_2025[interleaved_2025['def_rank'] > 15].reset_index(drop=True)

In [33]:
unders_2024

,player,runs,team,def_rank,games_played,1_runs_count,2_runs_count,3_runs_count,4_runs_count
0,Francisco Lindor,1.0,NYY,6.0,1,1,0,0,0
1,Ben Rice,0.0,NYM,15.0,1,0,0,0,0
2,Juan Soto,1.0,NYY,6.0,1,1,0,0,0
3,Aaron Judge,0.0,NYY,6.0,1,0,0,0,0
4,Pete Alonso,0.0,CHC,10.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
12553,MJ Melendez,0.0,ATL,2.0,1,0,0,0,0
12554,Hunter Renfroe,1.0,ATL,2.0,1,1,0,0,0
12555,Freddy Fermin,0.0,ATL,2.0,1,0,0,0,0
12556,Maikel Garcia,0.0,ATL,2.0,1,0,0,0,0


In [34]:
#Staacking unders on top of each other
unders_freq = pd.concat(
    [unders_2024, unders_2025],
    axis=0,
    ignore_index=True
)

In [35]:
#stacking overs on top of each other
overs_freq = pd.concat(
    [overs_2024, overs_2025],
    axis=0,
    ignore_index=True
)

In [36]:
#making a new df that sums the 1_hit_count and 2_hit_count and games_played and then creates the season frequency for overs
overs_freq = overs_freq.groupby('player', as_index=False).agg({
    '1_runs_count': 'sum',
    '2_runs_count': 'sum',
    '3_runs_count': 'sum',
    '4_runs_count': 'sum',
    'games_played': 'sum'
})

# Add a column for hit rate aka freq
overs_freq['1_runs_rate'] = overs_freq['1_runs_count'] / overs_freq['games_played']
overs_freq['2_runs_rate'] = overs_freq['2_runs_count'] / overs_freq['games_played']
overs_freq['3_runs_rate'] = overs_freq['3_runs_count'] / overs_freq['games_played']
overs_freq['4_runs_rate'] = overs_freq['4_runs_count'] / overs_freq['games_played']

In [37]:
#making a new df that sums the 1_hit_count and 2_hit_count and games_played and then creates the season frequency for unders 
unders_freq = unders_freq.groupby('player', as_index=False).agg({
    '1_runs_count': 'sum',
    '2_runs_count': 'sum',
    '3_runs_count': 'sum',
    '4_runs_count': 'sum',
    'games_played': 'sum'
})

# Add a column for hit rate
unders_freq['1_runs_rate'] = unders_freq['1_runs_count'] / unders_freq['games_played']
unders_freq['2_runs_rate'] = unders_freq['2_runs_count'] / unders_freq['games_played']
unders_freq['3_runs_rate'] = unders_freq['3_runs_count'] / unders_freq['games_played']
unders_freq['4_runs_rate'] = unders_freq['4_runs_count'] / unders_freq['games_played']

In [38]:
overs_freq

,player,1_runs_count,2_runs_count,3_runs_count,4_runs_count,games_played,1_runs_rate,2_runs_rate,3_runs_rate,4_runs_rate
0,A.J. Puk,0,0,0,0,2,0.000000,0.000000,0.000000,0.000000
1,Aaron Judge,34,13,3,1,57,0.596491,0.228070,0.052632,0.017544
2,Aaron Schunk,7,2,0,0,17,0.411765,0.117647,0.000000,0.000000
3,Abraham Toro,2,0,0,0,13,0.153846,0.000000,0.000000,0.000000
4,Adael Amador,0,0,0,0,3,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
649,Zach McKinstry,23,2,0,0,45,0.511111,0.044444,0.000000,0.000000
650,Zach Neto,12,4,0,0,34,0.352941,0.117647,0.000000,0.000000
651,Zach Penrod,0,0,0,0,1,0.000000,0.000000,0.000000,0.000000
652,Zack Gelof,13,4,0,0,34,0.382353,0.117647,0.000000,0.000000


In [39]:
#checking if distrobition is correct, merging them on name to make sure they average less hits against better pitchers and vise versa
final_freq = overs_freq.merge(
    unders_freq,
    on='player',
    how='inner',
    suffixes=('_over', '_under')
)

In [40]:
#dropping all rows that do not meet the creteria: avg less hits against better teams, and more against worse teams
final_freq = final_freq[final_freq['1_runs_rate_under'] <= final_freq['1_runs_rate_over']].reset_index(drop=True)

In [41]:
final_freq

,player,1_runs_count_over,2_runs_count_over,3_runs_count_over,4_runs_count_over,games_played_over,1_runs_rate_over,2_runs_rate_over,3_runs_rate_over,4_runs_rate_over,1_runs_count_under,2_runs_count_under,3_runs_count_under,4_runs_count_under,games_played_under,1_runs_rate_under,2_runs_rate_under,3_runs_rate_under,4_runs_rate_under
0,A.J. Puk,0,0,0,0,2,0.000000,0.000000,0.000000,0.000000,0,0,0,0,1,0.000000,0.000000,0.000000,0.000000
1,Aaron Judge,34,13,3,1,57,0.596491,0.228070,0.052632,0.017544,30,9,1,1,59,0.508475,0.152542,0.016949,0.016949
2,Aaron Schunk,7,2,0,0,17,0.411765,0.117647,0.000000,0.000000,4,1,0,0,23,0.173913,0.043478,0.000000,0.000000
3,Abraham Toro,2,0,0,0,13,0.153846,0.000000,0.000000,0.000000,1,0,0,0,14,0.071429,0.000000,0.000000,0.000000
4,Adam Frazier,18,3,0,0,44,0.409091,0.068182,0.000000,0.000000,7,0,0,0,39,0.179487,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,Zach DeLoach,4,0,0,0,5,0.800000,0.000000,0.000000,0.000000,3,1,0,0,7,0.428571,0.142857,0.000000,0.000000
426,Zach Dezenzo,7,5,0,0,17,0.411765,0.294118,0.000000,0.000000,8,0,0,0,25,0.320000,0.000000,0.000000,0.000000
427,Zach McKinstry,23,2,0,0,45,0.511111,0.044444,0.000000,0.000000,20,0,0,0,58,0.344828,0.000000,0.000000,0.000000
428,Zack Gelof,13,4,0,0,34,0.382353,0.117647,0.000000,0.000000,15,2,0,0,40,0.375000,0.050000,0.000000,0.000000


In [42]:
#dropping all players that do not have at least 10 games played in this category
final_freq = final_freq[
    (final_freq['games_played_over']  > 30) &
    (final_freq['games_played_under'] > 30)
].reset_index(drop=True)


In [43]:
# final_freq.to_csv('asdfasdfd.csv')

In [44]:
#making the final dataframes before sending them to the final script with odds
final_over_df=final_freq[['player','1_runs_rate_over','2_runs_rate_over','3_runs_rate_over','4_runs_rate_over']]
final_under_df=final_freq[['player','1_runs_rate_under','2_runs_rate_under','3_runs_rate_under','4_runs_rate_under']]

In [45]:
final_over_df

,player,1_runs_rate_over,2_runs_rate_over,3_runs_rate_over,4_runs_rate_over
0,Aaron Judge,0.596491,0.228070,0.052632,0.017544
1,Adam Frazier,0.409091,0.068182,0.000000,0.000000
2,Adley Rutschman,0.456522,0.065217,0.021739,0.000000
3,Adolis Garcia,0.338462,0.061538,0.000000,0.000000
4,Alejandro Kirk,0.288889,0.022222,0.000000,0.000000
...,...,...,...,...,...
169,Xavier Edwards,0.465116,0.046512,0.023256,0.000000
170,Yandy Diaz,0.410526,0.105263,0.010526,0.000000
171,Yordan Alvarez,0.454545,0.159091,0.022727,0.000000
172,Zach McKinstry,0.511111,0.044444,0.000000,0.000000


In [46]:
final_under_df

,player,1_runs_rate_under,2_runs_rate_under,3_runs_rate_under,4_runs_rate_under
0,Aaron Judge,0.508475,0.152542,0.016949,0.016949
1,Adam Frazier,0.179487,0.000000,0.000000,0.000000
2,Adley Rutschman,0.245283,0.018868,0.000000,0.000000
3,Adolis Garcia,0.142857,0.040816,0.000000,0.000000
4,Alejandro Kirk,0.175000,0.000000,0.000000,0.000000
...,...,...,...,...,...
169,Xavier Edwards,0.428571,0.053571,0.000000,0.000000
170,Yandy Diaz,0.293578,0.045872,0.000000,0.000000
171,Yordan Alvarez,0.276596,0.085106,0.021277,0.000000
172,Zach McKinstry,0.344828,0.000000,0.000000,0.000000


In [47]:
final_over_df.to_csv('overnewruns.csv')

In [48]:
final_under_df.to_csv('undernewruns.csv')